In [1]:
from memory_profiler import profile
import pennylane as qml
import pennylane.numpy as np
import time
from functools import wraps
from line_profiler import LineProfiler
import csv

In [3]:
# wires_m=[0]
# wires_k=[1]
# wires_solution=[2,3,4,5,6,7,8,9,10,11,12]

wires_m=[0,1]
wires_k=[2,3]
wires_solution=[4,5,6,7,8,9,10,11,12,13,14]

# wires_m=[0,1,2]
# wires_k=[3,4,5]
# wires_solution=[6,7,8,9,10,11,12,13,14,15,16]

# wires_m=[0,1,2,3]
# wires_k=[4,5,6,7]
# wires_solution=[8,9,10,11,12,13,14,15,16,17,18]
#
# wires_m=[0,1,2,3,4]
# wires_k=[5,6,7,8,9]
# wires_solution=[10,11,12,13,14,15,16,17,18,19,20,21]

# add_time=0
# mul_time=0
# total_time=0

In [11]:
dev = qml.device("default.qubit", wires=wires_m + wires_k + wires_solution, shots=1)
# dev = qml.device("qiskit.ibmq", wires=wires_m + wires_k + wires_solution, shots=1000, backend='',overwrite=True, ibmqx_token='')

n_wires = len(dev.wires)

In [ ]:
# def calculate_time_add(func):
#     @wraps(func)
#     def wrapper(*args, **kwargs):
#         start = time.time()
#         result = func(*args, **kwargs)
#         end = time.time()
#         execution_time = (end - start) * 1000  # 转换成毫秒
#         global add_time
#         add_time+=execution_time
#         return result
#     return wrapper
#
# def calculate_time_mul(func):
#     @wraps(func)
#     def wrapper(*args, **kwargs):
#         start = time.time()
#         result = func(*args, **kwargs)
#         end = time.time()
#         execution_time = (end - start) * 1000  # 转换成毫秒
#         global mul_time
#         mul_time+=execution_time
#         return result
#     return wrapper
#
# def calculate_time(func):
#     @wraps(func)
#     def wrapper(*args, **kwargs):
#         start = time.time()
#         result = func(*args, **kwargs)
#         end = time.time()
#         execution_time = (end - start) * 1000  # 转换成毫秒
#         global total_time
#         total_time=execution_time
#         return result
#     return wrapper

In [12]:
def mul_k_fourier(k,wi, wires):
    for j in range(len(wires)):
        qml.CRZ(k * np.pi / (2**j), wires=[wi,wires[j]])


def add_k_fourier(k, wires):
    for j in range(len(wires)):
        qml.RZ(k * np.pi / (2**j), wires=wires[j])

In [13]:
def multiplication(wires_m, wires_k, wires_solution):
    for i in range(len(wires_k)):
        for j in range(len(wires_m)):
            coeff = 2 ** (len(wires_m) + len(wires_k) - i - j - 2)
            qml.ctrl(mul_k_fourier, control=[wires_k[i]])(coeff,wires_m[j], wires_solution)


# @calculate_time_mul
@qml.qnode(dev)
def mul(m, k,wires_m, wires_k, wires_solution):
    qml.BasisEmbedding(m, wires=wires_m)
    qml.BasisEmbedding(k, wires=wires_k)

    qml.QFT(wires=wires_solution)

    multiplication(wires_m, wires_k, wires_solution)

    qml.adjoint(qml.QFT)(wires=wires_solution)
    return qml.sample(wires=wires_solution)

In [14]:
# @calculate_time_add
@qml.qnode(dev)
def sum3(m, k):
    qml.BasisEmbedding(m, wires=range(len(wires_solution)))  # m encoding
    qml.QFT(wires=range(len(wires_solution)))  # step 1
    add_k_fourier(k, range(len(wires_solution)))  # step 2
    qml.adjoint(qml.QFT)(wires=range(len(wires_solution)))  # step 3
    return qml.sample(wires=range(len(wires_solution)))

In [15]:
k=2
val=2
matrix_a=np.full((k,k),val)
matrix_b=np.full((k,k),val)

n=matrix_a.shape[0]
m=matrix_b.shape[1]
matrix_c=np.zeros((n,m))

In [16]:
def dot(a,b):
    listx=[]
    x=0
    for i in range(a.shape[0]):
        mulnum=mul(int(a[i]),int(b[i]),wires_m, wires_k, wires_solution)
        mulnum=int(''.join(map(str, list(mulnum))), 2)
        listx.append(mulnum)
    i=0
    while(listx.__len__()>1):
        i=0
        sumnum=sum3(listx[i], listx[i+1])
        sumnum=int(''.join(map(str, list(sumnum))), 2)
        x=x+sumnum
        i=i+1
        if listx.__len__()>=2:               #9 21
            listx=listx[2:]
            listx.insert(0,sumnum)
    return listx[0]


# @calculate_time
def matrix_multiplication_old():
    for i in range(n):
        for j in range(m):
            matrix_c[i, j] = dot(matrix_a[i], matrix_b[:, j])
            # print(matrix_c[i,j])
            print(matrix_c)


In [17]:
if __name__=='__main__':
    matrix_multiplication_old()

    # csv_file = open('multication_old.csv', 'a', newline='')
    # writer = csv.writer(csv_file)
    # data = [['size', 'value', 'add_time', 'mul_time', 'total_time'], [k, val, add_time, mul_time, total_time]]
    # writer.writerows(data)
    # csv_file.close()

[[8. 0.]
 [0. 0.]]
[[8. 8.]
 [0. 0.]]
[[8. 8.]
 [8. 0.]]
[[8. 8.]
 [8. 8.]]
